In [ ]:
import random
import pandas as pd
from statistics import mean
import plotly.express as px
import numpy as np

In [ ]:
def merge(l, r, comparisons):
    final = []
    left_counter = 0
    right_counter = 0
    while right_counter < len(r) and left_counter < len(l):
        comparisons[l[left_counter]] += 1
        comparisons[r[right_counter]] += 1
        if l[left_counter] <= r[right_counter]:
            final.append(l[left_counter])
            left_counter += 1
        else:
            final.append(r[right_counter])
            right_counter += 1
    if right_counter == len(r):
        final = final + l[left_counter:]
    else:
        final = final + r[right_counter:]
    return final

def mergeSort(x, comparisons):
    if len(x) == 2:
        comparisons[min(x)] += 1
        comparisons[max(x)] += 1
        return [min(x), max(x)]
    elif len(x) == 1:
        return x
    else:
        midpoint = int(len(x)/2)
        l = mergeSort(x[:midpoint], comparisons)
        r = mergeSort(x[midpoint:], comparisons)
    return merge(l, r, comparisons)

In [ ]:
class Team():
    def __init__(self, name, rank):
        self.name = name
        self.rank = rank
        self.expectedResult = None
        self.finalResult = None
    
    def __repr__(self):
        return f"Team {self.name}"

class Game():
    def __init__(self, team1, team2):
        self.team1 = team1
        self.team2 = team2
        self.result = self.GetWinnerAndLoser()

    def GetWinnerAndLoser(self):
        if self.team1.rank < self.team2.rank:
            return {"Winner": self.team1, "Loser": self.team2}
        else:
            return {"Winner": self.team2, "Loser": self.team1}
    
    def __repr__(self):
        return str(self.result)

class Tournament():
    def __init__(self, teams):
        self.teams = teams
        self.result = None
        self.rounds = int(np.log2(len(self.teams)))

        # random initial bracket
        self.order = random.sample(self.teams, len(self.teams))
        self.ranking = []
        self.comparisons = {t: 0 for t in self.teams}
    
    def FindWinner(self, displayResult = False):
        # Set where the teams finished in the previous knockout bracket
        if self.result is not None:
            finalResult = {t:k for k in self.result.keys() for t in self.result[k]}
            for t in self.teams:
                t.finalResult = finalResult[t]
        
        order = self.order.copy()
        level = {_: [] for _ in range(self.rounds+1)}

        for r in range(self.rounds):
            next_level = []

            # For each pair of teams, simulate a game between them
            for i in range(0, len(order), 2):
                G = Game(order[i], order[i + 1])

                # Add the loser to the list of losers this round
                level[r].append(G.result['Loser'])

                # next_round is the list of teams that advance to the next round
                next_level.append(G.result['Winner'])

                # If no tournament has been run, all comparisons are new
                if self.result is None: 
                    self.comparisons[G.result['Loser']] += 1
                    self.comparisons[G.result['Winner']] += 1
                # If the Loser was removed from the bracket, it doesn't count as a new game.
                elif G.result['Loser'].rank == float("inf"):
                    pass
                # If the winner, didn't advance as far before, it must be a new comparison
                elif G.result['Winner'].finalResult <= r:
                    self.comparisons[G.result['Loser']] += 1
                    self.comparisons[G.result['Winner']] += 1
                # If the loser didn't lose in this round before, then it must be a new comparison
                elif G.result['Loser'] not in self.result[r]:
                    self.comparisons[G.result['Loser']] += 1
                    self.comparisons[G.result['Winner']] += 1
            order = next_level.copy()

        # After all the rounds are done, the last team left is the winner
        level[self.rounds] = order.copy()
        self.result = level
        if displayResult:
            print(self.result)
        winner = level[self.rounds][0]
        self.ranking.append(winner)
        winner.rank = float("inf")

        return winner

    def tournamentSort(self):
        while len(self.ranking) < len(self.teams):
            self.FindWinner()

### MergeSort

In [ ]:
result = {"n": [], "Most Comparisons": [], "Least Comparisons": []}
m = 500
for n in range(100, 1000, 100):
    mins = []
    maxes = []
    for _ in range(m):
        test = list(range(n))
        comparisons = {x: 0 for x in test}
        random.shuffle(test)
        mergeSort(test, comparisons)
        mins.append(min(comparisons.values()))
        maxes.append(max(comparisons.values()))
    result["n"].append(n)
    result["Most Comparisons"].append(mean(maxes))
    result["Least Comparisons"].append(mean(mins))

In [ ]:
df = pd.DataFrame(result)
df = df.set_index("n")
display(df)
px.line(df, title = "Average Number of Comparisons by Element", labels = {"value": "Comparisons"})

In [ ]:
n = 1000
m = 250
mins = []
maxes = []
for _ in range(m):
    test = list(range(n))
    comparisons = {x: 0 for x in test}
    random.shuffle(test)
    mergeSort(test, comparisons)
    mins.append(min(comparisons.values()))
    maxes.append(max(comparisons.values()))

In [ ]:
display(px.histogram(pd.Series(maxes, name = "Max Comparisons"), title = "Max Number of Comparisons by Element"))

In [ ]:
n = 1024
test = list(range(n))
comparisons = {x: 0 for x in test}
random.shuffle(test)
mergeSort(test, comparisons)

px.histogram(pd.Series(comparisons, name = "Comparisons"), title = "Distribution of Comparisons by Element in MergeSort")

### Tournament Sort

In [ ]:
result = {"n": [], "Most Comparisons": [], "Least Comparisons": []}
m = 250

for i in range(7, 11):
    n = 2**i
    mins = []
    maxes = []
    for _ in range(m):
        teams = []
        for i in range(n):
            teams.append(Team(i, i))
        T = Tournament(teams)
        T.tournamentSort()
        mins.append(min(T.comparisons.values()))
        maxes.append(max(T.comparisons.values()))
    result["n"].append(n)
    result["Most Comparisons"].append(mean(maxes))
    result["Least Comparisons"].append(mean(mins))

In [ ]:
df = pd.DataFrame(result)
df = df.set_index("n")
display(df)
px.line(df, title = "Average Number of Comparisons by Element", labels = {"value": "Comparisons"})

In [ ]:
n = 1024
m = 250
mins = []
maxes = []
for _ in range(m):
        teams = []
        for i in range(n):
            teams.append(Team(i, i))
        T = Tournament(teams)
        T.tournamentSort()
        mins.append(min(T.comparisons.values()))
        maxes.append(max(T.comparisons.values()))

In [ ]:
display(px.histogram(pd.Series(maxes, name = "Max Comparisons"), title = "Max Number of Comparisons by Element"))

In [ ]:
n = 1024
teams = []
for i in range(n):
    teams.append(Team(i, i))
T = Tournament(teams)
T.tournamentSort()

px.histogram(pd.Series(T.comparisons, name = "Comparisons"), title = "Distribution of Comparisons by Element in Tournament Sort")